In [8]:
import os
import rasterio

import numpy as np

from rasterio.windows import Window

# Mount Inference Data

In [4]:
!sudo mount -t nfs4 -o nfsvers=4.1,rsize=1048576,wsize=1048576,hard,timeo=600,retrans=2,noresvport 172.31.91.151:/ ./efs_inference_data

# Windowing

In [9]:
def get_windows(img_dim, patch_size=(240, 240), stride=(240, 240)):
    patch_size = np.array(patch_size)
    stride = np.array(stride)
    img_dim = np.array(img_dim)
    
    max_dim = (img_dim//patch_size)*patch_size - patch_size

    ys = np.arange(0, img_dim[0], stride[0])
    xs = np.arange(0, img_dim[1], stride[1])

    tlc = np.array(np.meshgrid(ys, xs)).T.reshape(-1, 2)
    tlc = tlc[tlc[:, 0] <= max_dim[0]]
    tlc = tlc[tlc[:, 1] <= max_dim[1]]
    
    windows = []
    for y,x in tlc.astype(int):
        windows.append(Window(x, y, patch_size[1], patch_size[0]))

    return windows

In [12]:
def output_windows(path,out_path,patch_size=100,stride=100,SAVE=False,SAVE_INDIVIDUAL=False):

    raster = rasterio.open(path)
    src = raster.read()
    windows = get_windows(raster.shape, (patch_size, patch_size), (stride, stride))

    for j, window in enumerate(windows):

        data = raster.read((1,3,5),window=window, masked=True)
        
        print(data)

#         print(j, window, data.shape)

    #     if np.any(data.mask):
            # THE PATCH CONTAINS NODATA PIXELS, HANDLE AS NEEDED

    #     data = DO_PREPROCESSING(data.data)
    #     result = DO_INFERENCE(data)

        if SAVE:

            if SAVE_INDIVIDUAL:
                meta = raster.meta.copy()
                # Get the window specific transform - IF we want to save windows independantly
        #         trans = raster.window_transform(window)
                meta.update({
        #                 'transform': trans,
                        'dtype': src.dtype
                    })

                with rasterio.open(f"{out_path}/some_chip_{j}.tif", 'w', **meta) as dest:
                    dest.write(data)
            else:
                meta = raster.meta.copy()

                with rasterio.open(f"{out_path}/some_chip_{j}.tif", 'w+', **meta) as dest:
                    dest.write(data, window=window)

In [13]:
path = "./efs_inference_data/100_full_congo_export_v12_all_bands_Feb_10_19_20_29_2021.tif"
out_path = "./windows/"

output_windows(path,out_path)

[[[ 910  556  542 ...  501  501  501]
  [   0  394  328 ...  309  309  309]
  [   0  644  328 ...  309  309  309]
  ...
  [   0    0    0 ...  283  283  283]
  [   0    0    0 ...  283  283  283]
  [   0    0    0 ...  283  283  283]]

 [[1087  780  766 ...  721  737  720]
  [   0  648  586 ...  559  607  565]
  [   0  863  564 ...  579  553  537]
  ...
  [   0    0    0 ...  523  506  539]
  [   0    0    0 ...  552  562  529]
  [   0    0    0 ...  555  573  525]]

 [[1353 1058 1058 ...  984  998  998]
  [   0  900  844 ...  802  826  826]
  [   0 1152  861 ...  802  826  826]
  ...
  [   0    0    0 ...  824  825  825]
  [   0    0    0 ...  824  825  825]
  [   0    0    0 ...  801  797  797]]]
[[[ 492  492  492 ...  546  546  546]
  [ 310  310  310 ...  565  565  565]
  [ 310  310  310 ...  565  565  565]
  ...
  [ 284  284  284 ...  775  775  775]
  [ 284  284  284 ...  775  775  775]
  [ 276  276  276 ...  654  654  654]]

 [[ 713  711  733 ...  811  811  855]
  [ 554  558  546 